In [1]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess
import librosa
import numpy as np

from IPython.display import Audio as IPythonAudio

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (5.0.0)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Notice: ffmpeg is not installed. torchaudio is used to load audio
If you want to use ffmpeg backend to load audio, please install it by:
	sudo apt install ffmpeg # ubuntu
	# brew install ffmpeg # mac


In [2]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import io

# Parameters for recording
duration = 5  # seconds
sample_rate = 22050  # Hz, the default for librosa or your model



ModuleNotFoundError: No module named 'sounddevice'

In [19]:
#Record audio
print("Recording...")
audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
sd.wait()  # Wait until the recording is finished
print("Recording complete!")

# Convert the recorded audio to a format compatible with the model
# Here we create an in-memory buffer to simulate a file-like object
audio_buffer = io.BytesIO()

# Write the recorded audio to this in-memory buffer using soundfile
sf.write(audio_buffer, audio_data, sample_rate, format='mp3')

# Move to the beginning of the buffer so it can be read from the start
audio_buffer.seek(0)

# Read the audio data from the buffer (in memory) and ensure dtype is float32
audio_array, sr = sf.read(audio_buffer, dtype='float32')

# Display information about the processed audio data
print(f"Audio Array Dtype: {audio_array.dtype}, Shape: {audio_array.shape}, Sample Rate: {sr}")

# The audio data is now ready to be used directly with your transcription model
# For example:
#


Recording...
Recording complete!
Audio Array Dtype: float32, Shape: (110250,), Sample Rate: 22050


In [11]:
audio_path = r'C:\Users\Administrator\Desktop\Backend-LLM\audio_text_llm\SenseVoiceSmall\audio_sample\听故事学中文 Learn Chinese with 12 Stories - The Easiest Way to Improve Chinese.mp3'
audio_array, sampling_rate = librosa.load(audio_path, sr = None)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20036\3891720155.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_array, sampling_rate = librosa.load(audio_path, sr = None)
c:\Users\Administrator\Desktop\Backend-LLM\audio_text_llm\SenseVoiceSmall\sensevoice_env\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [26]:
# Print details
print("Sample Rate:", sampling_rate)
print("Audio Array:", audio_array)
print("Audio Array datatype is Array: ", isinstance(audio_array,np.ndarray) )

Sample Rate: 44100
Audio Array: [0. 0. 0. ... 0. 0. 0.]
Audio Array datatype is Array:  True


In [27]:
# Calculate the number of samples per chunk
chunk_samples = int(30 * sampling_rate)
total_samples = int(len(audio_array))

# Manual Splitting method

In [31]:
audio_path = r'C:\Users\Administrator\Desktop\Backend-LLM\audio_text_llm\SenseVoiceSmall\audio_sample\听故事学中文 Learn Chinese with 12 Stories - The Easiest Way to Improve Chinese.mp3'
audio_array, sampling_rate = librosa.load(audio_path, sr = None)
# List to hold transcriptions
transcriptions = []
model = AutoModel(
    model='paraformer-zh',
    # init_param = pretrained_model_path
    )
for start in range(0, total_samples, chunk_samples):
    end = min(start + chunk_samples, total_samples)
    chunk = audio_array[start:end]

    # Convert chunk to 16kHz if necessary
    # Note: Ensure your model accepts the sample rate of 16kHz or resample if needed
    chunk_resampled = librosa.resample(chunk, orig_sr = sampling_rate, target_sr = 16000)

    # Process the chunk
    res = model.generate(
        input=chunk_resampled,
        cache={},
        language="auto",
        use_itn=True,
        batch_size_s=60,
        merge_vad=True,
        merge_length_s=15,
    )

    # Get the transcribed text
    text = rich_transcription_postprocess(res[0]["text"])
    transcriptions.append(text)
    # Combine all transcriptions into one
full_transcription = " ".join(transcriptions)
# Save the transcription to a text file
with open("transcription_'paraformer-zh.txt", "w", encoding="utf-8") as file:
    file.write(full_transcription)


funasr version: 1.1.6.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.1.6


2024-09-06 15:50:25,189 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
c:\Users\Administrator\Desktop\Backend-LLM\audio_text_llm\SenseVoiceSmall\sensevoice_env\Lib\site-packages\funasr\train_utils\load_pretrained_model.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case w

In [28]:
# Resampling the Audio so it matches to the requirement of the model
audio_16KHz = librosa.resample(audio_array,
                               orig_sr=sampling_rate,
                
                               target_sr=16000)

# paraformer-zh model with vad_model


In [21]:
model = AutoModel(
    model='paraformer-zh',
     vad_model="fsmn-vad",
     vad_kwargs={"max_single_segment_time": 30000},
    device="cuda:0",
    #punc_model = "ct-punc"
)

# en
res = model.generate(
    input=audio_array,
    cache={},
    language="zn",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)
# # Save the transcription to a text file
# with open("./outdir/transcription_paraformer-zh.txt", "w", encoding="utf-8") as file:
#     file.write(full_transcription)

funasr version: 1.1.6.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.1.6


2024-09-12 16:43:37,907 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2024-09-12 16:43:47,312 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
  0%|          | 0/1 [00:00<?, ?it/s]

# SenseVoiceSmall Model with vad model

In [ ]:
model = AutoModel(
    model='iic/SenseVoiceSmall',
     vad_model="fsmn-vad",
     vad_kwargs={"max_single_segment_time": 30000},
    device="cuda:0",
    #punc_model = "ct-punc"
)

# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="zn",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)